In [65]:
import matplotlib
import networkx as nx
import classes as c

In [66]:
def generateNodeString(position, cf, ctp):
  return '[' + str(position) + ',' + str(cf) + ',' + str(ctp) + ']'

## Pruning Functons

In [67]:
def addNodeAndEdge(G, melodyData, prevIndex, posOverride = False):
  finalIndex = len(melody) - 1
  nextIndex = prevIndex + 1
  
  melody = melodyData['melody']
  prev1 = melody[prevIndex]
  
  if melody[nextIndex]:
    next1 = melody[prevIndex + 1]
    next2s = melodyData[str(next1)]
    if posOverride:
      prev2s = posOverride
    else:
      prev2s = melodyData[str(next1)]
  elif(prevIndex == finalIndex):
    prev2s = posOverride
  else:
    next1 = None
    next2 = None
    prev2s = []
    next2s = []
  for prevCtp in prev2s:
    prevStr = generateNodeString(prevIndex, prev1, prevCtp)
    validMoves = []
    if(prevIndex < finalIndex):
      for next2 in next2s:
        if(c.isValidMove(prev1, next1, prevCtp, next2)):
          validMoves.append(next2)
      G.add_node(prevStr)
      for next2 in validMoves:
        newNode = generateNodeString(nextIndex, next1, next2)
        G.add_node(newNode)
        G.add_edge(prevStr, newNode)

In [68]:
def removeIsolates(G):
  isolates = list(nx.isolates(G))
  G.remove_nodes_from(isolates)

# TODO make the function find path to actual final node(s), not just the last node that exists
# TODO find out what makes this function too strong for purpose -- removes too many nodes
def xTheNode(G, finalNote, finalIndex, finalNodes):
  strings = []
  remove = []
  for item in finalNodes:
    string = generateNodeString(finalIndex, finalNote, item)
    strings.append(string)
    for node in G.nodes:
      for item in strings:
        if G.has_node(item):
          isPath = nx.has_path(G, str(node), str(item))
          print("node: " + node + " final: " + item)
          if(not strings.__contains__(node)):
            if not isPath:
              remove.append(node)
              print(str(node) + ' failed')
          else:
            print(str(node))
  for node in remove:
    G.remove_node(node)

def pruneLeaves(G, finalIndex):
  # pseudo code
  # get degrees for all nodes
  degrees = G.degree
  remove = [node for node,degree in dict(G.degree()).items() if degree < 2]
  for node in remove:
    # isolate index
    index = node.replace("[", "").split(",")[0]
    print(index)
    print(int(index) < finalIndex)
    #if string[0] is not the final index
    if(int(index) < finalIndex):
      # if string[0] is not 0
      if(index != 0):
        # remove all nodes with degree of 1
        print(node)
        G.remove_node(node)

# potentially remove edges not connected to nodes

In [69]:
def removeNodes(G, finalNote, finalIndex, finalNodes):
  print(finalIndex)
  removeIsolates(G)
  xTheNode(G, finalNote, finalIndex, finalNodes)
  pruneLeaves(G, finalIndex)
  # too strong

In [ ]:
# check if moves are valid
def nextMoves(prev1, next1, prev2, posNext2):
  

In [70]:
def constructInitialGraph(melodyData, willPrint = False):
  melody = melodyData['melody']
  G = nx.DiGraph()
  """ nx.draw_spectral(G) """
  curIndex = 0
  finalIndex = len(melody) - 1
  
  # generate first nodes
  note = melody[0]
  if melodyData['ctpIsAbove']:
    firstPosCtp = {note + 0, note + 4, note +7, note + 12, note + 16, note + 19}
  else:
    firstPosCtp = {note, note - 12}
  nextMoves = melodyData[str(melody[1])]
  
  addNodeAndEdge(G, melodyData, curIndex, firstPosCtp)
  
  #generate middle nodes
  for i in range(1, finalIndex - 1):
    note = melody[i]
    print(" cur: " + str(curIndex) + " fin: " + str(finalIndex))
    nextCf = curIndex + 1
    nextMoves = []
    nextMoves = melodyData[str(melody[nextCf])]
    validMoves = []
    for possCtp in melodyData[str(note)]:
      possString = generateNodeString(curIndex, note, possCtp)
      for move in nextMoves:
        isValid = c.isValidMove(note, melody[nextCf], possCtp, move)
        if(isValid):
          validMoves.append(move)
      if validMoves:
        moveString = generateNodeString(curIndex + 1, melody[curIndex + 1], possCtp)
        G.add_node(moveString)
        for move in validMoves:
          G.add_edge(possString, moveString)
      else:
        if G.has_node(possString):
          G.remove_node(possString)
  
  # generate last nodes
  note = melody[finalIndex]
  if melodyData['ctpIsAbove']:
    finalCtp = {note, note + 12}
  else:
    finalCtp = {note, note - 12}
  
  
  addNodeAndEdge(G, melodyData, curIndex, finalCtp)
  removeNodes(G, melody[finalIndex], finalIndex, finalCtp)
  removeNodes(G, melody[finalIndex], finalIndex, finalCtp)
  
  if willPrint:
    print(G.nodes)
    print(G.edges)
    
  return G

In [71]:
def visualize(G):
  nx.random_layout(G)
  nx.draw_shell(G, with_labels = True)
  matplotlib.pyplot.figure(figsize = (2^16,2^16))

In [ ]:
from classes import getAllCombos

# only goes to index 2
combos = getAllCombos([0,5,4,2], True)
print(combos)
G = constructInitialGraph(combos, True)

visualize(G)
print(G)